**FINALIZED** **MODEL**

Libraries Required

In [2]:
import nltk #for building programs for text analysis(The Natural Language Toolkit)
from nltk.corpus import stopwords #getting the stopwords from nltk corpus function
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re # regular expression, helps us to match or find other strings or sets of strings
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
nltk.download('wordnet')
from nltk.corpus import wordnet
import textblob#perform basic NLP tasks.
from textblob import TextBlob
from textblob import Word
import string 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report#To get the classification report such as precison,recall(sesitivity),f1 score, support.. 
from sklearn.feature_extraction.text import CountVectorizer#Used to convert collection of text documents to a matrix of token counts.
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer#term frequencey- inverse document frequncy is a numerical statistic that is intended to reflect how important a word is to document in a collecion or corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading the csv file

In [3]:
from google.colab import files 
uploaded = files.upload()

Saving Hotel_Reviews_Ratings.csv to Hotel_Reviews_Ratings.csv


In [4]:
hotel=pd.read_csv("Hotel_Reviews_Ratings.csv")#Loading the dataset

hotel.head()

,reviews,ratings
0,We have stayed at this excellent hotel before ...,5
1,We had a fantastic overnight staycation to rec...,5
2,We chose this hotel for an overnight stay base...,3
3,"Really nice hotel, staff were very helpful and...",5
4,We stayed here for a week in September and rea...,5


Last 10 observations

In [5]:
hotel.tail(10)

,reviews,ratings
1990,"Basically, this place should be better than it...",3
1991,My wife and I spent two nights at the Hotel De...,5
1992,We stayed for a week between 20-27th July 2012...,3
1993,Nice hotel with good bars and resteraunts room...,5
1994,Hotel de France is set back from the main road...,4
1995,This was our third stay at this lovely hotel a...,5
1996,We visited in early July and stayed for 5 nigh...,5
1997,"took my wife here for 5 nights, had a spa room...",4
1998,"Booked a double room. When I arrived, checked...",1
1999,Great Hotel. Was there for 3 nights. Staff w...,5


In [6]:
#Classifiying the rating into positive, Negative, Neutral 
#Rating which is greater than 3 considered as +ve, Less than 3 considered as -ve, And 3 is considered as Neutral.
Sentiment_Rating = [] 
for value in hotel["ratings"]: 
    if value > 3: 
        Sentiment_Rating.append(1) 
    elif value < 3: 
        Sentiment_Rating.append(2) 
    else: 
        Sentiment_Rating.append(0) 

In [7]:
hotel["Sentiment_Rating"] = Sentiment_Rating    #adding the classified column to the dataset
print(hotel)

                                                reviews  ...  Sentiment_Rating
0     We have stayed at this excellent hotel before ...  ...                 1
1     We had a fantastic overnight staycation to rec...  ...                 1
2     We chose this hotel for an overnight stay base...  ...                 0
3     Really nice hotel, staff were very helpful and...  ...                 1
4     We stayed here for a week in September and rea...  ...                 1
...                                                 ...  ...               ...
1995  This was our third stay at this lovely hotel a...  ...                 1
1996  We visited in early July and stayed for 5 nigh...  ...                 1
1997  took my wife here for 5 nights, had a spa room...  ...                 1
1998  Booked a double room.  When I arrived, checked...  ...                 2
1999  Great Hotel.  Was there for 3 nights.  Staff w...  ...                 1

[2000 rows x 3 columns]


In [8]:
hotel.head(10)

,reviews,ratings,Sentiment_Rating
0,We have stayed at this excellent hotel before ...,5,1
1,We had a fantastic overnight staycation to rec...,5,1
2,We chose this hotel for an overnight stay base...,3,0
3,"Really nice hotel, staff were very helpful and...",5,1
4,We stayed here for a week in September and rea...,5,1
5,During the Covid 19 crisis the trick for hotel...,5,1
6,Stayed at this hotel 5 times in 7 months for b...,5,1
7,This hotel offered outstanding service from st...,5,1
8,We stayed for a week in July and wasn't disapp...,5,1
9,14 of us spent the most wonderful afternoon at...,5,1


#########preprocesssing the reviews######################


In [9]:

#converting the review to lower case
hotel['Reviews']= hotel['reviews'].apply(lambda x: " ".join(word.lower() for word in x.split()))

#removing the punctuations from the reviews
hotel['Reviews']=hotel['Reviews'].apply(lambda x:''.join([i for i in x  if i not in string.punctuation]))

#removing the numericals from the reviews
hotel['Reviews']=hotel['Reviews'].str.replace('[0-9]','')

#removing all the stopwords from the reviews
stop_words=stopwords.words('english')
hotel['Reviews']=hotel['Reviews'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

#Applying Lemmitaization to the reviews
hotel['Reviews']= hotel['Reviews'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
#removing the unwanted symbols from the reviews
pattern = r"((?<=^)|(?<= )).((?=$)|(?= ))"
hotel['Reviews']= hotel['Reviews'].apply(lambda x:(re.sub(pattern, '',x).strip()))

Model using TFIDF Vectorizer with RandomForest Classifier

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer#term frequencey- inverse document frequncy is a numerical statistic that is intended to reflect how important a word is to document in a collecion or corpus
# Fit the TfidfVectorizer to the training data
Tfidf = TfidfVectorizer().fit(hotel.Reviews)
# transform the documents in the training data to a document-term matrix
Tfidf_vect = Tfidf.transform(hotel.Reviews).toarray()

Saving the Tfidf Vectorizer into pickle file

In [20]:
import pickle
##Saving the countvectorizer
filename = 'Tfidf_file.pkl'
pickle.dump(Tfidf, open(filename, 'wb'))

In [11]:
#Applying the smoting technique to level the imbalanced scenario.
#pip install imblearn
from imblearn.over_sampling import SMOTE 
over_sample= SMOTE(random_state= 50, sampling_strategy= "all")
X_over,Y_over=over_sample.fit_sample(Tfidf_vect,hotel["Sentiment_Rating"])

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [12]:
#spliting the data's into train and test
Xtrain,Xtest,Ytrain,Ytest= train_test_split(X_over,Y_over,test_size = 0.2,random_state=0)

Training the model and predicting the result

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=2,oob_score=True,n_estimators=1000,criterion="entropy")
rf.fit(Xtrain,Ytrain)
Rf_Prediction_tfidf=rf.predict(Xtest)

In [14]:
print("accuracy_score: ", f1_score(Ytest,Rf_Prediction_tfidf,average='micro'))

accuracy_score:  0.9839924670433146


In [15]:
#Classification Report
print("Classification Report \n\n\n", classification_report(Ytest,Rf_Prediction_tfidf))

Classification Report 


               precision    recall  f1-score   support

           0       1.00      0.96      0.98       358
           1       0.96      1.00      0.98       352
           2       1.00      1.00      1.00       352

    accuracy                           0.98      1062
   macro avg       0.98      0.98      0.98      1062
weighted avg       0.98      0.98      0.98      1062



confusion_matrix

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Ytest,Rf_Prediction_tfidf)
print (confusion_matrix)

[[343  15   0]
 [  1 351   0]
 [  0   1 351]]


#prediting the output for a single review


In [18]:
review = ["hotel undergoing refurbishment effect made clear website say refurbishment complete end march fail say restaurant bar closed offered garden view restaurant better cafeteria part restaurant overlook garden full junk bar simply part spa area expect four star hotel dont book unless understand situation least refurb complete"]
predicted = Tfidf.transform(review)
rf.predict(predicted)


array([2])

The model is performing well, Because it predicted as negative result which is also marked as negative in the dataset. And 2 is considered as Negative, 0 is Neutral and 1 is positive

Saving the model using Pickle module

In [19]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(rf, open(filename, 'wb'))